In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
import tls_client

In [2]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
requests = tls_client.Session(
    client_identifier="chrome112",
)
response1 = requests.get(dog_props_url)

data = response1.json()

In [3]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [4]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
2430,0c5c5b7a-69ed-4324-aa76-50246a108a2b,False,None,[{'id': '77b9ac91-382c-475f-9a51-281d5c6d5d07'...,da8d116a-a17f-409d-8ef2-84be05b59ecd,25102049,1.5,active,da8d116a-a17f-409d-8ef2-84be05b59ecd,aa7583d3-521b-46ce-813a-1fc4e34d4d55,a1f85405-f23f-44d7-a12a-4a5c2bcd0958,Kills in Map 2,high_score,8d5d17e8-9672-4da2-bc19-29be1d6815c1,kills_in_map_2,None,None,LoL: Nisqy Kills in Map 2 O/U
3219,32280c86-02e1-4e3e-a2ed-2050be94e40b,False,None,[{'id': 'f30b1eb9-5799-4f6b-bc19-0ef3108c8e12'...,15c9f075-4f09-4ea7-aecc-ffa8ed34bed3,25191151,28.0,active,15c9f075-4f09-4ea7-aecc-ffa8ed34bed3,94cc62af-5a2d-4a8b-bda0-3e2e6b3cf659,c40ba2b5-6ac9-4371-bf5d-b4790117219e,Kills on Map 1+2,high_score,fd4d222d-d4d0-4258-b0d2-d225e1947b0c,map_1_and_2_kills,None,None,CS: oxygeN Kills on Map 1+2 O/U


In [5]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [6]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [7]:
player_df

,full_name,sport_id,team_id
0,LeBron James,NBA,97616c91-0509-42f8-99f3-04f3856a6377
1,Anthony Davis,NBA,97616c91-0509-42f8-99f3-04f3856a6377
2,Austin Reaves,NBA,97616c91-0509-42f8-99f3-04f3856a6377
3,D'Angelo Russell,NBA,97616c91-0509-42f8-99f3-04f3856a6377
4,Rui Hachimura,NBA,97616c91-0509-42f8-99f3-04f3856a6377
...,...,...,...
881,C. Carroll + K. Marte,MLBCOMBOS,db69f844-d6eb-4332-b133-796ce316890a
882,C. Walker + T. Pham,MLBCOMBOS,db69f844-d6eb-4332-b133-796ce316890a
883,N. Castellanos + A. Bohm,MLBCOMBOS,17bb8634-d7d6-4a42-af6e-603155ca1f8f
884,K. Schwarber + J. Realmuto,MLBCOMBOS,17bb8634-d7d6-4a42-af6e-603155ca1f8f


In [8]:
player_df.sport_id.unique()

array(['NBA', 'NFL', 'MLB', 'NHL', 'CFB', 'NBASZN', 'FIFA', 'ESPORTS',
       'NBACOMBOS', 'MLBCOMBOS'], dtype=object)

In [9]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [10]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
81,56777.0,59e326fc-9b55-44f9-8a17-42215e654b86,0.0,3c703313-0b48-4938-86ae-d173a9e3d9f4,0.0,Tue 02:00pm,0.0,2023-10-24T18:00:00Z,regular,ESPORTS,scheduled,CS2: 1WIN vs. Sampi,Game,2023.0
82,56778.0,a1db780b-d55f-4e69-8de6-0fb03b60b3ec,0.0,130a1ea4-f142-4ca3-bf0e-83242c450157,0.0,Tue 02:00pm,0.0,2023-10-24T18:00:00Z,regular,ESPORTS,scheduled,CS2: Corinthians vs. Boca Juniors,Game,2023.0
83,56801.0,5c8eddf2-bc26-457b-a097-97a3b8cbef85,0.0,e95e00cd-f7ca-46a6-b1a2-ea45dd75a199,0.0,Tue 01:00pm,0.0,2023-10-24T17:00:00Z,regular,ESPORTS,scheduled,CS2: JANO vs. ENCE Academy,Game,2023.0


In [11]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [12]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [13]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [14]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,52356.0,97616c91-0509-42f8-99f3-04f3856a6377,0.0,5d478d55-84f7-4786-beb9-9c009c5814f4,0.0,Tue 07:40pm,0.0,2023-10-24,regular,NBA,scheduled,LAL @ DEN,Game,2023.0
1,52357.0,f98f3baa-ad03-4188-9e8a-0cb52e1b900e,0.0,d399490e-0df0-4c33-b7c2-fafdadc3a4eb,0.0,Tue 10:10pm,0.0,2023-10-24,regular,NBA,scheduled,PHX @ GSW,Game,2023.0
2,52360.0,8f5852b6-c8dc-4cce-980a-eab09b28919a,0.0,c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e,0.0,Wed 07:10pm,0.0,2023-10-25,regular,NBA,scheduled,BOS @ NYK,Game,2023.0
3,52361.0,09540cc3-fc27-4a59-b7a6-7a63f0e1df95,0.0,27c02dfc-343a-432b-8a64-c963035b609d,0.0,Wed 07:10pm,0.0,2023-10-25,regular,NBA,scheduled,HOU @ ORL,Game,2023.0
4,52359.0,402685bd-a73c-4eb1-8c0f-55debd622215,0.0,4da4781a-a2c2-49d7-a042-1dee71740668,0.0,Wed 07:10pm,0.0,2023-10-25,regular,NBA,scheduled,WAS @ IND,Game,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,56776.0,5d60f0ef-dc86-4b82-ba86-83df490e7b45,0.0,50a4445a-c860-4937-9846-a5d3d5297c0f,0.0,Tue 01:30pm,0.0,2023-10-24,regular,ESPORTS,scheduled,CS2: FTW vs. Entropiq,Game,2023.0
80,56724.0,501eaaff-16cc-4df5-8f76-f0f82f3b136c,0.0,01549315-0767-40a6-b89a-8f4631ff69cc,0.0,Tue 02:00pm,0.0,2023-10-24,regular,ESPORTS,scheduled,IBCP: UCAM Tokiers vs. Ramboot Club,Game,2023.0
81,56777.0,59e326fc-9b55-44f9-8a17-42215e654b86,0.0,3c703313-0b48-4938-86ae-d173a9e3d9f4,0.0,Tue 02:00pm,0.0,2023-10-24,regular,ESPORTS,scheduled,CS2: 1WIN vs. Sampi,Game,2023.0
82,56778.0,a1db780b-d55f-4e69-8de6-0fb03b60b3ec,0.0,130a1ea4-f142-4ca3-bf0e-83242c450157,0.0,Tue 02:00pm,0.0,2023-10-24,regular,ESPORTS,scheduled,CS2: Corinthians vs. Boca Juniors,Game,2023.0


In [15]:
id2tm

{'5d478d55-84f7-4786-beb9-9c009c5814f4': 'DEN',
 'd399490e-0df0-4c33-b7c2-fafdadc3a4eb': 'GSW',
 'c71a1a59-28ee-4ac6-ba44-d8aeb0d2ac3e': 'NYK',
 '27c02dfc-343a-432b-8a64-c963035b609d': 'ORL',
 '4da4781a-a2c2-49d7-a042-1dee71740668': 'IND',
 '11cfe154-8ba6-4c22-be8f-2365656fb4da': 'CHA',
 'a34e5fa9-e4aa-4195-98bf-05710d97347d': 'BKN',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92': 'MIA',
 'fd41e35e-7b5d-48c3-be0c-69e958976f6b': 'TOR',
 '8ee0305d-067f-45e5-8586-1c111d3a43ec': 'CHI',
 '56a5d8c8-eb72-47db-9e93-5b24d8f5443a': 'MEM',
 'ade6b8fc-178f-4e7e-b43d-aa853bea0f3a': 'UTA',
 'fee4a894-1970-402a-873b-8faea88f4ec9': 'SAS',
 'bd4b60cd-5adb-4cc6-8e8a-97084501f386': 'LAC',
 'e142b705-dd0a-4803-8cde-a35788a65341': 'MIL',
 '0719b253-43db-532e-8b5c-6c12c6c3f951': 'BUF',
 '26e67e06-664e-50a6-ad7b-c102705fde8b': 'DAL',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 'a6d8dc19-daaf-5798-a8f2-df7f9fc9eecd': 'WAS',
 '9153e0a9-da83-5246-ba43-417537f1bcce': 'MIA',
 'f11f1cf1-9933-5203-8181-95020ee64399':

In [16]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [17]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['LeBron James', 'Anthony Davis', 'Austin Reaves',
       "D'Angelo Russell", 'Rui Hachimura', 'Taurean Prince',
       'Gabe Vincent', 'Christian Wood', 'Nikola Jokic', 'Jamal Murray',
       'Michael Porter Jr.', 'Aaron Gordon', 'Kentavious Caldwell-Pope',
       'Christian Braun', 'Kevin Durant', 'Devin Booker', 'Jusuf Nurkic',
       'Stephen Curry', 'Klay Thompson', 'Chris Paul', 'Andrew Wiggins',
       'Kevon Looney', 'Jayson Tatum', 'Jaylen Brown',
       'Kristaps Porzingis', 'Jrue Holiday', 'Derrick White',
       'Jalen Brunson', 'Julius Randle', 'RJ Barrett',
       'Mitchell Robinson', 'Quentin Grimes', 'Fred VanVleet',
       'Jalen Green', 'Jabari Smith Jr.', 'Alperen Sengun',
       'Dillon Brooks', 'Paolo Banchero', 'Franz Wagner',
       'Wendell Carter Jr.', 'Markelle Fultz', 'Jordan Poole',
       'Tyus Jones', 'Daniel Gafford', 'Tyrese Haliburton',
       'Myles Turner', 'Bruce Brown', 'Bennedict Mathurin', 'Obi Toppin',
       'Trae Young', 'Dejounte Murray'

In [18]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NBASZN', 'NBASZN', 'NBASZN', 'NBASZN', 'NBASZN']

In [19]:
guys[1000:1007]

['Zach LaVine',
 'Zach LaVine',
 'Nikola Vucevic',
 'Nikola Vucevic',
 'Nikola Vucevic',
 'Nikola Vucevic',
 'Nikola Vucevic']

In [20]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['b075de5a-578f-417c-8a5c-cf0ff08101b2',
 'b075de5a-578f-417c-8a5c-cf0ff08101b2',
 'b075de5a-578f-417c-8a5c-cf0ff08101b2',
 'b075de5a-578f-417c-8a5c-cf0ff08101b2',
 'b075de5a-578f-417c-8a5c-cf0ff08101b2']

In [21]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,23.5,points,LeBron James Points O/U,LeBron James,NBASZN
1,38.5,pts_rebs_asts,LeBron James Pts + Rebs + Asts O/U,LeBron James,NBASZN
2,6.5,assists,LeBron James Assists O/U,LeBron James,NBASZN
3,7.5,rebounds,LeBron James Rebounds O/U,LeBron James,NBASZN
4,14.5,rebs_asts,LeBron James Rebounds + Assists O/U,LeBron James,NBASZN
...,...,...,...,...,...
3323,2.5,total_bases,N. Castellanos + A. Bohm Total Bases O/U,N. Castellanos + A. Bohm,other
3324,3.5,hits_runs_rbis,C. Walker + T. Pham Hits + Runs + RBIs O/U,C. Walker + T. Pham,other
3325,3.5,hits_runs_rbis,N. Castellanos + A. Bohm Hits + Runs + RBIs O/U,N. Castellanos + A. Bohm,other
3326,12.5,fantasy_points,L. Gurriel Jr. + G. Moreno Fantasy Points O/U,L. Gurriel Jr. + G. Moreno,other


In [22]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,23.5,points,LeBron James Points O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2
1,38.5,pts_rebs_asts,LeBron James Pts + Rebs + Asts O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2
2,6.5,assists,LeBron James Assists O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2
3,7.5,rebounds,LeBron James Rebounds O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2
4,14.5,rebs_asts,LeBron James Rebounds + Assists O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2
...,...,...,...,...,...,...
3323,2.5,total_bases,N. Castellanos + A. Bohm Total Bases O/U,N. Castellanos + A. Bohm,other,17bb8634-d7d6-4a42-af6e-603155ca1f8f
3324,3.5,hits_runs_rbis,C. Walker + T. Pham Hits + Runs + RBIs O/U,C. Walker + T. Pham,other,db69f844-d6eb-4332-b133-796ce316890a
3325,3.5,hits_runs_rbis,N. Castellanos + A. Bohm Hits + Runs + RBIs O/U,N. Castellanos + A. Bohm,other,17bb8634-d7d6-4a42-af6e-603155ca1f8f
3326,12.5,fantasy_points,L. Gurriel Jr. + G. Moreno Fantasy Points O/U,L. Gurriel Jr. + G. Moreno,other,db69f844-d6eb-4332-b133-796ce316890a


In [23]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [24]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,23.5,points,LeBron James Points O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
1,38.5,pts_rebs_asts,LeBron James Pts + Rebs + Asts O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
2,6.5,assists,LeBron James Assists O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
3,7.5,rebounds,LeBron James Rebounds O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
4,14.5,rebs_asts,LeBron James Rebounds + Assists O/U,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
...,...,...,...,...,...,...,...
3343,2.5,total_bases,N. Castellanos + A. Bohm Total Bases O/U,N. Castellanos + A. Bohm,other,17bb8634-d7d6-4a42-af6e-603155ca1f8f,NaN
3344,3.5,hits_runs_rbis,C. Walker + T. Pham Hits + Runs + RBIs O/U,C. Walker + T. Pham,other,db69f844-d6eb-4332-b133-796ce316890a,NaN
3345,3.5,hits_runs_rbis,N. Castellanos + A. Bohm Hits + Runs + RBIs O/U,N. Castellanos + A. Bohm,other,17bb8634-d7d6-4a42-af6e-603155ca1f8f,NaN
3346,12.5,fantasy_points,L. Gurriel Jr. + G. Moreno Fantasy Points O/U,L. Gurriel Jr. + G. Moreno,other,db69f844-d6eb-4332-b133-796ce316890a,NaN


In [25]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [26]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,23.5,points,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
1,38.5,pts_rebs_asts,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
2,6.5,assists,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
3,7.5,rebounds,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
4,14.5,rebs_asts,LeBron James,NBASZN,b075de5a-578f-417c-8a5c-cf0ff08101b2,NaN
...,...,...,...,...,...,...
3343,2.5,total_bases,N. Castellanos + A. Bohm,other,17bb8634-d7d6-4a42-af6e-603155ca1f8f,NaN
3344,3.5,hits_runs_rbis,C. Walker + T. Pham,other,db69f844-d6eb-4332-b133-796ce316890a,NaN
3345,3.5,hits_runs_rbis,N. Castellanos + A. Bohm,other,17bb8634-d7d6-4a42-af6e-603155ca1f8f,NaN
3346,12.5,fantasy_points,L. Gurriel Jr. + G. Moreno,other,db69f844-d6eb-4332-b133-796ce316890a,NaN


In [27]:
df = df.reset_index(drop=True)

In [28]:
df.Stat.unique()

array(['points', 'pts_rebs_asts', 'assists', 'rebounds', 'rebs_asts',
       'pts_rebs', 'pts_asts', 'free_throws_made', 'turnovers',
       'three_points_att', 'personal_fouls', 'blks_stls',
       'field_goals_att', 'steals', 'three_points_made', 'blocks',
       'double_doubles', 'passing_yds', 'receiving_yds', 'rushing_yds',
       'passing_ints', 'fantasy_points', 'strikeouts', 'pitch_outs',
       'runs_allowed', 'hits_runs_rbis', 'singles', 'total_bases',
       'hits_allowed', 'runs', 'goals_against', 'saves', 'shots',
       'blocked_shots', 'goals', 'passing_and_rushing_yds',
       'receiving_rec', 'ppg', 'rpg', 'apg', '3pg', 'shots_attempted',
       'goals_assists', 'chances_created', 'shots_on_target', 'passes',
       'fouls_against', 'aces', 'double_faults', 'sets_lost',
       'breakpoints_won', 'service_games_lost', 'games_won', 'games_lost',
       'games_played', 'kills_in_maps_1_2', 'fantasy_points_on_maps_1_2',
       'fantasy_points_on_map_1', 'kills_in_map_1',
 

In [29]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [30]:
df = df.replace(syntax)

In [31]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
2269,14.0,kills_in_maps_1_2,Berserker+Emenes,ESPORTS,Cloud9,2023-10-26
2270,10.5,kills_in_maps_1_2,Blaber+Fudge,ESPORTS,Cloud9,2023-10-26
2271,42.4,fantasy_points_on_maps_1_2,Fudge,ESPORTS,Cloud9,2023-10-26
2272,21.6,fantasy_points_on_map_1,Fudge,ESPORTS,Cloud9,2023-10-26
2273,5.5,kills_in_maps_1_2,Fudge,ESPORTS,Cloud9,2023-10-26
...,...,...,...,...,...,...
3306,15.5,headshots_on_maps_1_2,myltsi,ESPORTS,ENCE Academy,2023-10-24
3307,9.0,headshots_on_maps_1_2,podi,ESPORTS,ENCE Academy,2023-10-24
3308,12.5,headshots_on_maps_1_2,HENU,ESPORTS,ENCE Academy,2023-10-24
3309,12.5,headshots_on_maps_1_2,S1rva,ESPORTS,ENCE Academy,2023-10-24


In [32]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date
3107,33.5,MAPS 1-2 Kills,Krimbo,ESPORTS,BIG,2023-10-24
3108,16.5,headshots_on_maps_1_2,Krimbo,ESPORTS,BIG,2023-10-24


In [33]:
today = dt.date.today()

In [34]:
str(today)

'2023-10-24'

In [35]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [36]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
2269,14.0,kills_in_maps_1_2,Berserker+Emenes,ESPORTS,Cloud9,2023-10-26
2270,10.5,kills_in_maps_1_2,Blaber+Fudge,ESPORTS,Cloud9,2023-10-26
2271,42.4,fantasy_points_on_maps_1_2,Fudge,ESPORTS,Cloud9,2023-10-26
2272,21.6,fantasy_points_on_map_1,Fudge,ESPORTS,Cloud9,2023-10-26
2273,5.5,kills_in_maps_1_2,Fudge,ESPORTS,Cloud9,2023-10-26
...,...,...,...,...,...,...
3306,15.5,headshots_on_maps_1_2,myltsi,ESPORTS,ENCE Academy,2023-10-24
3307,9.0,headshots_on_maps_1_2,podi,ESPORTS,ENCE Academy,2023-10-24
3308,12.5,headshots_on_maps_1_2,HENU,ESPORTS,ENCE Academy,2023-10-24
3309,12.5,headshots_on_maps_1_2,S1rva,ESPORTS,ENCE Academy,2023-10-24
